# (4) Define the AFA problem (action options and costs)
To properly specify the AFA problem, the costs and available actions have to be defined. 

In static settings, we do currently only support 
- misclassification cost (cost if the prediction is wrong, defined by a cost matrix) 
- feature acquisition cost (cost for acquiring a certain feature, defined by a vector) 

We currently do not allow other costs, e.g.:
- time to diagnosis cost (when we acquire more feature, we might delay the diagnosis which produces a cost) 

In **time-series** settings, we additionally require:
- resolution (e.g. 6 hours) which describes the intervals at which actions can be taken.
- delay (e.g. 2 hours) which describes after which time the acquired features are available. 

In [1]:
%load_ext autoreload
%autoreload 2

### Define paths 

In [2]:
# which dataset to work on 
dataset_name   = "synthetic_2"

In [3]:
# data specifications 
data_dir            = "../../../data/ts/" + dataset_name + "/fully_observed/" 

afa_problem_files = {
        'action_file'    : data_dir + 'afa_problem/' + 'acquisition_actions.csv', 
        'prediction_cost_file'    : data_dir + 'afa_problem/' + 'prediction_cost.csv'
                }

## Part 1: Define Action options (and acquisition costs) 

### Option 1: Define costs directly via .csv files
Fill a .csv file ('acquisition_actions.csv') by 
- listing action names as columnnames
- filling the second row with the acquisition cost of each acquisition action 
- filling the third row with the resolution of the respective action (e.g. every 6 hours) 
- listing in the subsequent rows all superfeature names that are acquired by the respective action 

### Option 2: Define the action options here and save the mapping 
A second option is to define the action options as a dictionary of dictionaries and save it.  
Format:  
action_options :
    dictionary defining the action options (keys of dict), each action is a dict with 
- 'cost', a float value for the cost
- 'target_superfeatures', list of strings
    the names of the superfeatures that can be acquired with one action 
- for time-series  
    - 'resolution', a float values that says after how many time steps this action can be taken again
    - 'delay (e.g. 2 hours) which describes after which time the acquired features are available. 

In [4]:
from afa.afa_problem.utils import save_action_options

2023-03-21 11:22:47.305396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 11:22:47.391387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 11:22:47.391403: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 11:22:50.090718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
action_options = \
    { 'acquire_superX1_ts' : {
                            'cost' : 0.5, 
                            'target_superfeatures' : ['superX1_ts'], 
                            'resolution' : 2, 
                            'delay': 1
                            }, 
      'acquire_superX2_ts' : {
                            'cost' : 1, 
                            'target_superfeatures' : ['superX2_ts'],
                            'resolution' : 2, 
                            'delay': 0
                            }
    }

save_action_options( action_options = action_options , action_file = afa_problem_files['action_file'] )  

### Option 3: Prepared action options (not recommended)
Lastly, for specific datasets, the action option creation can be automatically executed. This is however not recommended. 

In [6]:
from afa.configurations.data_settings.define_data_settings_ts import generate_action_options_ts

action_options = generate_action_options_ts( dataset_name ,  data_dir = data_dir )

## Part 2: Define prediction costs (for misclassification)

### Option 1: Define costs directly via .csv files
Fill a .csv file ('prediction_costs.csv') by 
- listing 'predicted Y = 0', 'predicted Y = 1', ... as column names 
- listing 'true Y = 0', 'true Y = 1', ... as rownames 
- filling the the resulting matrix with the costs
-> entry(i+1,j+1) is the cost of predicting a datapoint of class i as belonging to class j 

### Option 2: Define the misclassification costs here and save the mapping 
A second option is to define the misclassification matrix here directly and save it as a .csv file  
Format:  
C_mc:  numpy array  
- entry(i,j) is the cost of predicting a datapoint of class i as belonging to class j 
    

In [6]:
from afa.afa_problem.utils import save_prediction_cost
import numpy as np

In [7]:
C_mc = np.array(
    [[0, 10], 
     [10, 0]]
)

save_prediction_cost( C_mc = C_mc , prediction_cost_file = afa_problem_files['prediction_cost_file'] )  

### Option 3: Prepared action options (not recommended)
Lastly, for specific datasets, the misclassification cost definition can be automatically executed. This is however not recommended.

In [9]:
from afa.configurations.data_settings.define_data_settings_ts import generate_prediction_cost_ts

C_mc = generate_prediction_cost_ts( dataset_name ,  data_dir = data_dir )

## Test by loading the AFA problem

In [8]:
from afa.afa_problem.afa_problem_loader.afa_problem_loader import AFAProblemLoader

In [9]:
afa_problem_loader = AFAProblemLoader(   action_file          = afa_problem_files['action_file'], 
                                         prediction_cost_file = afa_problem_files['prediction_cost_file'])
afa_problem = afa_problem_loader.load() 

In [10]:
# list action options
afa_problem.action_options

{'acquire_superX1_ts': {'target_superfeatures': ['superX1_ts'],
  'cost': 0.5,
  'resolution': 2.0,
  'delay': 1.0},
 'acquire_superX2_ts': {'target_superfeatures': ['superX2_ts'],
  'cost': 1.0,
  'resolution': 2.0,
  'delay': 0.0}}

In [11]:
# list misclassification costs 
afa_problem.C_mc

array([[ 0, 10],
       [10,  0]])

## Explain the created AFA Problem 
Write a summary of the defined AFA problem to file (latex or markdown) 

In [12]:
# reporting
explanation_file = data_dir + 'afa_problem/' +  'afa_problem_report'
afa_problem.explain(file= explanation_file + '.md'  , format='markdown')
# afa_problem.explain(file= explanation_file + '.tex' , format='latex')

FileNotFoundError: [Errno 2] No such file or directory: 'pandoc'